In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

In [2]:
top100 = pd.read_csv('../data/RawYearlyTop100.csv')
top100.head()

,Rank,Song,Artist(s),Year
0,1,Theme from A Summer Place,Percy Faith,1960
1,2,He'll Have to Go,Jim Reeves,1960
2,3,Cathy's Clown,The Everly Brothers,1960
3,4,Running Bear,Johnny Preston,1960
4,5,Teen Angel,Mark Dinning,1960


In [3]:
top100["Artist(s)"] = top100["Artist(s)"].str.split('featuring', expand=True)[0]
top100["Song"] = top100["Song"].str.replace("'", " ")
top100["Query"] = "track:" + top100["Song"] + " artist:" + top100["Artist(s)"]
top100.head()

,Rank,Song,Artist(s),Year,Query
0,1,Theme from A Summer Place,Percy Faith,1960,track:Theme from A Summer Place artist:Percy F...
1,2,He ll Have to Go,Jim Reeves,1960,track:He ll Have to Go artist:Jim Reeves
2,3,Cathy s Clown,The Everly Brothers,1960,track:Cathy s Clown artist:The Everly Brothers
3,4,Running Bear,Johnny Preston,1960,track:Running Bear artist:Johnny Preston
4,5,Teen Angel,Mark Dinning,1960,track:Teen Angel artist:Mark Dinning


In [4]:
with open('../config.json') as file:
    app_data = json.load(file)

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials( \
                     client_id=app_data['client_id'], \
                     client_secret=app_data['client_secret']))

In [26]:
years = range(1966,1981)
for year in years:
    this_year = top100[top100['Year'] == year]
    new_cols = list(this_year.columns) + ['ID', 'Popularity', 'Release Date']
    results = pd.DataFrame(columns=new_cols)
    no_results = pd.DataFrame()

    for i, row in this_year.iterrows():
        search_res = sp.search(q=row['Query'], limit=5)
        tracks = search_res['tracks']['items']
        if len(tracks) == 0:
            no_results = no_results.append(row, ignore_index=True)
        for idx, track in enumerate(tracks):
            row['ID'] = track['id']
            row['Popularity'] = track['popularity']
            row['Release Date'] = track['album']['release_date']
            results = results.append(row, ignore_index=True)

    results.to_csv('../data/search-results/' + str(year) + '.csv', index=False)
    no_results.to_csv('../data/search-results/' + str(year) + '_missing.csv', index=False)